___

# [ Machine Learning in Geosciences ] 
Department of Applied Geoinformatics and Carthography, Charles University

Lukas Brodsky lukas.brodsky@natur.cuni.cz


## Python-ogr vectors


Thi notebook introduces how to work with vector data in Python using ogr/gdal binding. It covers: 

* 1. Reading vector data; 

* 2. Cycling over features; 

* 3. Creating a new data source and layer; 


### Setup

In [ ]:
# Common imports for reading, visualizing
import os
import sys
import matplotlib.pyplot as plt 
%matplotlib inline 

# import ogr
from osgeo import ogr


### Get data 

In [ ]:
# Set your own PATH!!! 
PATH = './data/'

if os.path.isdir(PATH): 
    print('ok')

In [ ]:
vector_fn = os.path.join(PATH, 'polygons.shp')

In [ ]:
if os.path.isfile(vector_fn): 
    print('ok')
else: 
    print('Check your path to the vector file!')

### Reading vector

In [ ]:
# get the shapefile driver
driver = ogr.GetDriverByName('ESRI Shapefile')

In [ ]:
driver

In [ ]:
# open the data source
datasource = driver.Open(vector_fn, 0)
if datasource is None:
  print('Could not open file')
  sys.exit(1)

In [ ]:
datasource

In [ ]:
# get metadata
datasource.GetLayerCount() 

In [ ]:
# get the data layer
layer = datasource.GetLayer()

In [ ]:
layer

In [ ]:
# loop through the features in the layer

feature = layer.GetNextFeature()
i = 0
while feature:
    i += 1
    print(i)
    # get the attributes
    note = feature.GetFieldAsString('note')
    print(note)
    
    # work with the geometry as you need
    # get the x,y coordinates
    geom = feature.GetGeometryRef()
    # x = str(geom.GetX())
    # y = str(geom.GetY())
    x_cntr = geom.Centroid().GetX()
    y_cntr = geom.Centroid().GetY()
    print(f"X centroid: {round(x_cntr, 2)}")
    print(f"Z centroid: {round(y_cntr, 2)}")
    
    # destroy the feature and get a new one
    feature.Destroy()
    feature = layer.GetNextFeature()


In [ ]:
# See more methods here: 
# http://pcjericks.github.io/py-gdalogr-cookbook/geometry.html 

In [ ]:
# close the data source and text file
datasource.Destroy()

### Creating vector (polygon centroids) 

In [ ]:
os.getcwd()

In [ ]:
# os.chdir(...)

In [ ]:
driver = ogr.GetDriverByName('ESRI Shapefile')


In [ ]:
# open the input data source and get the layer
inDS = driver.Open(vector_fn, 0)
if inDS is None:
  print('Could not open file')
  sys.exit(1)
inLayer = inDS.GetLayer()


In [ ]:
inLayer


In [ ]:
# create a new data source and layer

fn = os.path.join(PATH, 'polygon_centroid.shp') 
if os.path.exists(fn):
  driver.DeleteDataSource(fn)
outDS = driver.CreateDataSource(fn)
if outDS is None:
  print('Could not create file')
  sys.exit(1)
outLayer = outDS.CreateLayer(fn, geom_type=ogr.wkbPoint)

In [ ]:
# get the FieldDefn's for the existing fields in the input shapefile
feature = inLayer.GetFeature(0)
labelFieldDefn = feature.GetFieldDefnRef('label')
noteFieldDefn = feature.GetFieldDefnRef('note')


In [ ]:
labelFieldDefn


In [ ]:
# create new fields in the output shapefile
outLayer.CreateField(labelFieldDefn)
outLayer.CreateField(noteFieldDefn)


In [ ]:
# get the FeatureDefn for the output layer
featureDefn = outLayer.GetLayerDefn()


In [ ]:
# inFeature = inLayer.GetNextFeature()


In [ ]:
# geom = inFeature.GetGeometryRef()


In [ ]:
# centroid = geom.Centroid()


In [ ]:
# print(centroid.GetX(), centroid.GetY())


In [ ]:
# loop through the input features
inFeature = inLayer.GetNextFeature()
while inFeature:
    # get the cover attribute for the input feature
    label = inFeature.GetField('label')
    
    # create a new feature
    outFeature = ogr.Feature(featureDefn)

    # set the geometry
    geom = inFeature.GetGeometryRef()
    centroid = geom.Centroid()
    centroid.GetX()
    centroid.GetY()
    
    outFeature.SetGeometry(centroid)

    # set the attributes
    note = inFeature.GetField('note')
    outFeature.SetField('label', label)
    outFeature.SetField('note', note)

    # add the feature to the output layer
    outLayer.CreateFeature(outFeature)

    # destroy the output feature
    outFeature.Destroy()
    
    # destroy the input feature and get a new one
    inFeature.Destroy()
    inFeature = inLayer.GetNextFeature()

In [ ]:
# close the data sources
inDS.Destroy()
outDS.Destroy()
